# Setup for Db2 Event Store Order Details Database

This notebook uses the [Scala](https://www.scala-lang.org/) programming language
to interact with IBM Db2 Event Store.

1. Connect to IBM Db2 Event Store
2. Create a database
3. Define a table schema
4. Create a table
5. Query the table

## Connect to IBM Db2 Event Store

### Determine the IP address of your host

Obtain the IP address of the host that you want to connect to by running the appropriate command for your operating system:

* On Mac, run: `ifconfig`
* On Windows, run: `ipconfig`
* On Linux, run: `hostname -i`

Edit the `HOST = "XXX.XXX.XXX.XXX"` value in the next cell to provide the IP address.

In [ ]:
// Set your host IP address
var host = "XXX.XXX.XXX.XXX"

//Port will be 1100 for version 1.1.2 or later (5555 for version 1.1.1)
var port = "1100"

// Database name
var db = "TESTDB"

// Table name
var tableName = "OnlineRetailOrderDetail"
var cancelTableName = "OnlineRetailCancelDetail"

## Import Scala packages

In [ ]:
import sys.process._
import scala.concurrent.{Await, Future}
import scala.concurrent.duration.Duration
import collection.JavaConverters._
import org.apache.spark.sql.Row
import org.apache.spark.sql.types._
import org.apache.spark.sql.ibm.event.EventSession
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.DataFrameReader

import com.ibm.event.catalog.TableSchema
import com.ibm.event.oltp.EventContext
import com.ibm.event.example.DataGenerator
import com.ibm.event.common.ConfigurationReader
import com.ibm.event.oltp.InsertResult

## Connect to Event Store

In [ ]:
ConfigurationReader.setConnectionEndpoints(host + ":" + port)

## Create a database

Only one database can be active in Event Store. If you already have a database, you don't need to create one.
To create a database in Event Store, you can use the createDatabase function. If you want to drop an existing
database to create a new one, use the dropDatabase function first.

In [ ]:
// See the comments and run this cell if you need to DROP and/or CREATE the database.

// EventContext.dropDatabase(db)  // Uncomment this if you want to drop an existing TESTDB
var created = EventContext.createDatabase(db) // Comment this out to re-use an existing TESTDB
var error =  created.openDatabase()
error.map(e => sys.error(e.toString))

In [ ]:
EventContext.openDatabase(db)
var eventContext = EventContext.getEventContext

## Create a table

### Define the schema

In [ ]:
var orderDetailStruct = StructType(Array(
  StructField("id", LongType, nullable = false),
  StructField("InvoiceNo", LongType, nullable = false),
  StructField("StockCode", StringType, nullable = false),
  StructField("Description", StringType, nullable = false),
  StructField("Quantity", IntegerType, nullable = false),
  StructField("InvoiceDate", TimestampType, nullable = false),
  StructField("UnitPrice", DoubleType, nullable = false),
  StructField("CustomerId", StringType, nullable = false),
  StructField("Country", StringType, nullable = false)))

// Define table schema for order details
var tableSchema = TableSchema(
  tableName,
  orderDetailStruct,
  shardingColumns = Seq("InvoiceNo"),
  pkColumns = Seq("InvoiceNo", "id"))

// Define table schema for cancellation details
var cancelTableSchema = TableSchema(
  cancelTableName,
  orderDetailStruct,
  shardingColumns = Seq("InvoiceNo"),
  pkColumns = Seq("InvoiceNo", "id"))

### Create the Table
If you want to drop the existing table to create a new one, use the dropTable function first.

In [ ]:
// Create the tables - skip if tables are already created

// eventContext.dropTable(tableSchema.tableName)  // Uncomment to drop existing table
// eventContext.dropTable(cancelTableSchema.tableName)  // Uncomment to drop existing table
var res = eventContext.createTable(tableSchema)
if (res.isDefined) {
  println(s"Error while creating table ${tableSchema.tableName}\n: ${res.get}")
} else {
  println(s"Table ${tableSchema.tableName} successfully created.")
}
var res2 = eventContext.createTable(cancelTableSchema)
if (res2.isDefined) {
  println(s"Error while creating table ${cancelTableSchema.tableName}\n: ${res2.get}")
} else {
  println(s"Table ${cancelTableSchema.tableName} successfully created.")
}

## Load data from the CSV file to a DataFrame
Use the `add data assets` in the UI to make the file available to the notebook.
Then read the file from the assets directory into a DataFrame.

In [ ]:
// Initialize spark session
var spark = SparkSession.builder().getOrCreate()

## Load data from the DataFrame to the table
Use the batchInsert function to load the data.

In [ ]:
val sqlContext = new EventSession(spark.sparkContext, db)
sqlContext.loadEventTable(tableName).registerTempTable(tableName)
sqlContext.loadEventTable(cancelTableName).registerTempTable(cancelTableName)

var testDF = sqlContext.sql(s"select * from $tableName")
testDF.show(5)
var testCancelDF = sqlContext.sql(s"select * from $cancelTableName")
testCancelDF.show(5)

<p><font size=-1 color=gray>
&copy; Copyright 2018 IBM Corp. All Rights Reserved.
<p>
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file
except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the
License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either
express or implied. See the License for the specific language governing permissions and
limitations under the License.
</font></p>